# 新闻分类：多分类问题

## 路透社数据集

- 加载路透社数据集

In [1]:
from keras.datasets import reuters

Using TensorFlow backend.


In [2]:
(train_data,train_labels),(test_data,test_labels)=reuters.load_data(num_words=10000)

In [3]:
len(train_data)

8982

In [4]:
len(test_data)

2246

In [5]:
print(train_data[1])

[1, 3267, 699, 3434, 2295, 56, 2, 7511, 9, 56, 3906, 1073, 81, 5, 1198, 57, 366, 737, 132, 20, 4093, 7, 2, 49, 2295, 2, 1037, 3267, 699, 3434, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2295, 2, 2, 775, 7, 48, 34, 191, 44, 35, 1795, 505, 17, 12]


- 将索引解码为新闻文本

In [15]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i-3,'?') for i in train_data[10]])
decoded_newswire

'? period ended december 31 shr profit 11 cts vs loss 24 cts net profit 224 271 vs loss 511 349 revs 7 258 688 vs 7 200 349 reuter 3'

In [7]:
train_labels[10]

3

## 准备数据

In [16]:
import numpy as np

In [18]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences),dimension))
    for i,sequences in enumerate(sequences):
        results[i,sequences] = 1
    return results

- 将训练数据，测试数据向量化

In [19]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

将标签向量化有两种方法：
1. 将标签列表转换为整数张量
2. 使用one-hot编码（分类编码）

In [26]:
def to_one_hot(labels,dimension=46):
    results = np.zeros((len(labels),dimension))
    for i,label in enumerate(labels):
        results[i,label] = 1
    return results

- 将训练标签，测试标签向量化

In [27]:
one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

- 注意，Keras内置方法可以实现这个操作

In [28]:
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

## 构建网络

- 这个主题分类问题与前面的电影评论分类问题类似，两个问题都是试图对简短的文本片段进行分类。但这个问题有一个新的约束条件：输出类别的数量从2个变为46个。输出空间的维度要大得多。
- 对于前面用过的Dense层的堆叠，每层只能访问上一层输出的信息。如果某一层丢失了与分类问题相关的一些信息，那么这些信息无法被后面的层找回，也就是说，每一层都可能成为信息瓶颈。上一个例子使用了16维的中间层，但对这个例子来说16维空间可能太小了，无法学会区分46各不同类别。这种维度较小的层可能成为信息瓶颈，永久地丢失信息。
- 处于这个原因，下面将使用维度更大的层，包换64个单元。

### 模型定义